In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import string
from nltk.stem.snowball import SnowballStemmer

In [2]:
# Reading the data file into a DATAFRAME and checking the shape 
data=pd.read_csv('35.1AmazonMobileDataUncleaned.csv')
print(data.shape)

(71922, 2)


In [3]:
def datapreprocess(sen):
    
    sen = re.sub(r"didn't", "did not", sen)
    sen = re.sub(r"don't", "do not", sen)
    sen = re.sub(r"won't", "will not", sen)
    sen = re.sub(r"can't", "can not", sen)
    sen = re.sub(r"wasn't", "was not", sen)
    sen = re.sub(r"\'ve", " have", sen)
    sen = re.sub(r"\'m", " am", sen)
    sen = re.sub(r"\'ll", " will", sen)
    sen = re.sub(r"\'re", " are", sen)
    sen = re.sub(r"\'s", " is", sen)
    sen = re.sub(r"\'d", " would", sen)
    sen = re.sub(r"\'t", " not", sen)
    sen = re.sub(r"\'m", " am", sen)
    
    
    p = set(string.punctuation) # p take all the punctuations, punctuations displayed below
    sen=sen.lower() # convert text to lower case
    words=sen.split() # split the text into words
    ctext=[]
    for i in range(10): #adding numbers from 0 - 9 to p 
        p.add(str(i))
    for i in words:
        t=''.join([x for x in i.encode("ascii","ignore").decode("ascii") if x not in p]) # ignoring non ascii charecters and numbers
        ctext.append(t)
    return " ".join([i for i in ctext]) # joining the cleaned words to text
    
    

stop=set(stopwords.words('english'))
print(stop)
stop.remove('no')
stop.remove('not')
stop.remove('nor')

preprocessed_essays = []
for sentance in data['uncleanedreview'].values:
    try:
        ctxt=datapreprocess(str(sentance))
        if len(ctxt)==0: raise()
        ctxt = ' '.join(e for e in ctxt.split() if e not in stop)
        preprocessed_essays.append(ctxt.lower().strip())
    except:
        preprocessed_essays.append("NAN")  
data['cleanedtext'] = preprocessed_essays
data.drop(data[data["cleanedtext"]=="NAN"].index,axis = 0,inplace = True) # deleting rows that have no text 
data=data.reset_index(drop='true') # after deleting rows index should be reset
data['decision']= data['decision'].replace('positive',1)
data['decision']= data['decision'].replace('negative',0)


s = SnowballStemmer("english")
p=[]
for i in data['cleanedtext'].values:
    k=[]
    for word in i.split():
        k.append(s.stem(word))
    l=' '.join(k)
    p.append(l)

data['finaltext']=p

data.to_csv (r'AmazonMobileDataCleaned.csv')

{'all', 'o', 'at', 'very', 't', 're', 'him', "it's", 'their', 'can', 'wouldn', 'off', 'our', 'had', 'too', 'to', 'were', 'as', 'the', 'doing', 'above', 'when', 'into', "you're", 'those', 'been', 'on', 'below', 'most', "haven't", 'after', 'how', 'mightn', 'weren', "you'd", "couldn't", 'he', 'because', 'ourselves', "mightn't", 'am', 'have', 'them', 'that', 'should', 'does', 'herself', "aren't", "shan't", 'nor', 'wasn', 'shan', 'up', 'yourself', 'theirs', 'she', 'do', "hasn't", "doesn't", 'out', 'with', 'why', 'couldn', 'if', 'you', 'this', 'needn', 'same', 'i', 'hers', 'd', 'for', 'here', 'myself', 'll', 'in', 'doesn', 'so', 'themselves', 'aren', 'an', "didn't", "shouldn't", 'against', "needn't", 's', 'any', 'having', "mustn't", 'once', 'her', 'under', 'from', 'ours', 'yourselves', 'his', 'was', 'it', 'don', 'they', 'before', 'just', 'further', "isn't", 'itself', 'be', 'over', 'such', 'm', 'is', 'did', 'haven', "she's", 'through', "should've", 've', 'didn', 'me', 'until', 'both', 'a', 'a

In [5]:
data=pd.read_csv('AmazonMobileDataCleaned.csv')

label = data['decision'].values
feedback = data.drop(['decision'], axis=1)


from sklearn.model_selection import train_test_split
inputtrain, inputtest, outputtrain, outputtest = train_test_split(feedback, label, test_size=0.20, stratify=label)
inputtrain, inputcv, outputtrain, outputcv = train_test_split(inputtrain, outputtrain, test_size=0.20, stratify=outputtrain)
print(inputtrain.shape)
print(inputcv.shape)
print(inputtest.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)

(46029, 4)
(11508, 4)
(14385, 4)
(46029,)
(11508,)
(14385,)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(min_df=10,ngram_range=(1,4))
vec.fit(inputtrain['finaltext'].values) # fit has to happen only on train data


train_cleanedtext = vec.transform(inputtrain['finaltext'].values)
cv_cleanedtext = vec.transform(inputcv['finaltext'].values)
test_cleanedtext= vec.transform(inputtest['finaltext'].values)

print("After vectorizations")
print(train_cleanedtext.shape)
print(cv_cleanedtext.shape)
print(test_cleanedtext.shape)
print(outputtrain.shape)
print(outputcv.shape)
print(outputtest.shape)
print("...............................")

After vectorizations
(46029, 24338)
(11508, 24338)
(14385, 24338)
(46029,)
(11508,)
(14385,)
...............................
